**Install dependencies:** In case this notebook is not running [Carto-Lab Docker](https://cartolab.theplink.org/), the cell below aims to install the needed packages for this notebook. If packages are already available, they will be ignored.

In [8]:
import sys
pyexec = sys.executable
print(f"Current Kernel {pyexec}")
!../py/modules/pkginstall.sh "{pyexec}" geopandas

Current Kernel /opt/conda/envs/worker_env/bin/python3.13
geopandas already installed (version 1.0.1).


# Data Projection

The Earth is curved, but spatial data is often represented as flat maps. Converting from 3D to 2D causes stretching and inaccuracies in measurements like distance and area.
**Projections** help transform data into flat maps while minimizing distortions.

```{figure} ../resources/26.png
:width: 900px
:name: figure-example

Map projection (Source: [Wikipedia](https://en.wikipedia.org/wiki/Map_projection#/media/File:Usgs_map_miller_cylindrical.PNG))
````


## CRS (Coordinate Reference System)

A Coordinate Reference System (CRS) defines the location of spatial objects on the Earth’s surface.

There are two main types:

- `Geographic Coordinate Systems (GCS)`: Define locations using latitude and longitude on a spherical model of the Earth (e.g., WGS84).
- `Projected Coordinate Systems (PCS)`: Flatten the curved surface into a 2D map using Cartesian coordinates (e.g., UTM).

| Feature             | Geographic Coordinate System (GCS) | Projected Coordinate System (PCS) |
|----------------------|------------------------------------|------------------------------------|
| **Purpose** | Preserve shape over large areas    | Minimize distortions locally       |
| **Measurement Units**| Degrees (Latitude/Longitude)      | Meters or feet                     |
| **Best for** | Global-scale mapping             | Local/regional mapping            |
| **Common distortion**| Distance and area distortions     | Shape or scale distortions        |

Spatial relationships like joins and overlays require matching coordinate systems.
If datasets have different CRS, they must be reprojected, or else they will not align correctly, leading to incorrect analysis.

To manage projections and CRS, we use the `geopandas` package.

### Working with CRS in GeoPandas

In [9]:
from pathlib import Path
import geopandas as gp

# Define the input path
INPUT = Path.cwd().parents[0] / "00_data"
gdb_path = INPUT / "Biotopwerte_Dresden_2018.gdb"

# Load the data
input_data = gp.read_file(gdb_path, layer="Biotopwerte_Dresden_2018")

### Checking the current CRS

You can check the current coordinate system of the data using the `crs` attribute:

In [10]:
input_data.crs

<Projected CRS: EPSG:25833>
Name: ETRS89 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 12°E and 18°E: Austria; Croatia; Denmark - offshore and offshore; Germany - onshore and offshore; Italy - onshore and offshore; Norway including Svalbard - onshore and offshore.
- bounds: (12.0, 34.79, 18.01, 84.01)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

Key information: 

 - `Coordinate System`: ETRS89/UTM Zone 33N (accurate for Central Europe)
 - `Coordinates`: E and N represent Easting (X) and Northing (Y) in cartesian CRS
 - `Units`: Meters

`````{admonition} EPSG Code
:class: caution 

An **EPSG code** is a unique identifier for coordinate reference systems.
You can search for EPSG codes for your region using [EPSG.io from MapTiler](https://epsg.io/#google_vignette). 
`````

`````{admonition} Example: Geographic Coordinate System
:class: hint, dropdown

```{figure} ../resources/25.png
:width: 700px
:name: figure-example

Example of a Geographic Coordinate System
````

In this example, the key information are:
 - `Coordinate System`: WGS 84, a global CRS, covers the entire Earth 
 - `Coordinates`: Latitude and Longitude, representing Northing and Easting
 - `Unit of Measurements`: Degrees

A geographic CRS (such as WGS84) uses latitude and longitude in degrees. When using GCS for maps, the height (Z values) is typically ignored, resulting in a Geographic 2D CRS.

`````

### Assigning a CRS

If the `crs` output is `None`, it means the dataset does not have a CRS assigned.
You can manually set the correct CRS (based on metadata) using the `.set_crs()` method:

In [11]:
data = input_data.set_crs("EPSG:25833")

### Reprojecting the data

To transform the data into a different coordinate system (reprojection), use the `.to_crs()` method.

There are two ways to specify the target CRS:

**1. Using an EPSG code as a string**

In [12]:
data_2 = input_data.to_crs("EPSG:3035")
data_2.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

**2. Using an EPSG code as an integer**

In [13]:
data_3 = input_data.to_crs(epsg=3035)
data_3.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

**Matching another layer’s CRS** 

Sometimes you want one dataset to match another dataset’s CRS:

In [14]:
data_4 = data_3.to_crs(input_data.crs)
data_4.crs

<Projected CRS: EPSG:25833>
Name: ETRS89 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 12°E and 18°E: Austria; Croatia; Denmark - offshore and offshore; Germany - onshore and offshore; Italy - onshore and offshore; Norway including Svalbard - onshore and offshore.
- bounds: (12.0, 34.79, 18.01, 84.01)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

Now both layers are aligned and spatial operations (joins, overlays, measurements) can be performed correctly.